In [ ]:
#===========read sigle file===========
import pandas as pd
import ast

dictx = []

path = '/Users/rubythurs/Library/Application Support/Godot/app_userdata/Ray/save_2023-12-4-11.47.35.csv'
with open(path) as f:
    data = f.read()
    data2 = ast.literal_eval(data) #tuple
    for item in data2:
        if isinstance(item, dict):
            dictx.append(item)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(dictx)
dfg = df.dropna(subset=['grade'])
dfg
#==========end==========

In [ ]:
#=========================================

In [ ]:
#===========read from directory===========

import pandas as pd
import numpy as np
import ast
import os

# the data form is required to have no space at the beginning and nothing and the end,
# and dicts should be separated with comma.

#raw data files:
paths = []
directory = '/Users/rubythurs/PycharmProjects/count/datadoe/3f'
for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        filepath = os.path.join(directory,filename)
        paths.append(filepath)
len(paths)

In [ ]:
for i in range(0,len(paths)):
    print(paths[i])

In [ ]:
dictx = []
for i in range(0,len(paths)):
    path = paths[i]
    with open(path) as f:
        data = f.read()
        data2 = ast.literal_eval(data) #tuple
        for item in data2:
            if isinstance(item, dict):
                dictx.append(item)

pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(dictx)

dfg = df.dropna(subset=['grade'])
dfg

#===========end reading===========

In [ ]:
#unite the grades
mask =  dfg['ini_time'] == dfg['ini_time'].shift(-1)
dfg.loc[mask, 'grade'] = np.nan
dfgh = dfg.bfill()
# things to analyze
dfgh['response_time'] = dfgh['click_time']-dfgh['ini_time']
dfgh[['ini_x','ini_y']] = dfgh['ini_pos'].str.strip('()').str.split(',', expand=True)
dfgh[['click_x','click_y']] = dfgh['click_pos'].str.strip('()').str.split(',', expand=True)
# Convert the entire 'Column1' from string to numeric
columns_to_convert = ['click_x', 'click_y', 'ini_x','ini_y']
dfgh[columns_to_convert] = dfgh[columns_to_convert].apply(pd.to_numeric)
dfgh['group']= pd.cut(dfgh['percent_visible'],bins=[i/10 for i in range(11)], labels=[f'{i/10}-{(i+1)/10}' for i in range(10)])
dfgh

In [ ]:
dfgh.shape

In [ ]:
#================================

In [ ]:
#=========cor rate==========

In [ ]:
unique_groups = dfgh['group'].unique()
print(f'percent groups: {len(unique_groups)}')


# Create a dictionary to store the selected DataFrames
dfs = {} #all responses
dfss ={} # 1 response per trail

# Select rows for each unique value in the "Group" column
for group_value in unique_groups:
    dfs[group_value] = dfgh[dfgh['group'] == group_value]
    #correct rate
    latest_touch_time= dfs[group_value].groupby('ini_time')['response_time'].idxmax()
    dfss[group_value] = dfs[group_value].loc[latest_touch_time]
    q = dfss[group_value]['grade'].value_counts()[1]
    shape = dfss[group_value].shape[0]
    rate=q / shape
    
 

    print(f'{group_value},  total_touches: {dfs[group_value].shape[0]},  total_trails:{shape},  cor_rate: {rate.round(2)},')

In [ ]:
#=========time messages==========

In [ ]:
unique_groups = dfgh['group'].unique()
print(f'percent groups: {len(unique_groups)}')


# Create a dictionary to store the selected DataFrames
dfs = {} #all responses


# Select rows for each unique value in the "Group" column
for group_value in unique_groups:
    dfs[group_value] = dfgh[dfgh['group'] == group_value]    
    #time message
    loc1 = dfs[group_value].groupby('ini_time')['response_time'].idxmax()
    loc0 = dfs[group_value].groupby('ini_time')['response_time'].idxmin()
    mintime= dfs[group_value].loc[loc0]['response_time'].values * 0.001
    maxtime= dfs[group_value].loc[loc1].response_time.values *0.001
    duration = (maxtime-mintime)
    #print(f'{group_value},   {mintime.mean().round()},  {maxtime.mean().round()},  {duration.mean().round()}')
    print(f'{group_value},  {mintime.std().round()},  {maxtime.std().round()},  {duration.std().round()},')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt



dfgh.boxplot(column='response_time', by='group')

# Set the plot title and axis labels
plt.title('Box Plot')
plt.xlabel('Group')
plt.ylabel('Value')

# Display the plot
plt.show()

In [ ]:
unique_groups = dfgh['group'].unique()
print(f'percent groups: {len(unique_groups)}')


# Create a dictionary to store the selected DataFrames
dfs = {} #all responses

per = []
mintime_mean = []
maxtime_mean = []
duration_mean = []
mintime_std = []
maxtime_std = []
duration_std= []

# Select rows for each unique value in the "Group" column
for group_value in unique_groups:
    dfs[group_value] = dfgh[dfgh['group'] == group_value]   
    per.append(group_value)
    
    #time message
    loc1 = dfs[group_value].groupby('ini_time')['response_time'].idxmax()
    loc0 = dfs[group_value].groupby('ini_time')['response_time'].idxmin()
    mintime= dfs[group_value].loc[loc0]['response_time'].values * 0.001
    maxtime= dfs[group_value].loc[loc1].response_time.values *0.001
    duration = (maxtime-mintime)
    
    mintime_std.append(mintime.std().round())
    mintime_mean.append(mintime.mean().round())
    maxtime_mean.append(maxtime.mean().round())
    maxtime_std.append(maxtime.std().round())
    duration_mean.append(duration.mean().round())
    duration_std.append(duration.std().round())
  

dft = {'percent':per,'mintime_mean':mintime_mean,'mintime_std':mintime_std,}
dft = pd.DataFrame(dft)
dft

In [ ]:
#=========end time messages==========

In [ ]:
dfs['0.5-0.6']

In [ ]:
latest_touch_time= dfs['0.3-0.4'].groupby('ini_time')['response_time'].idxmax()
dfs4= dfs['0.3-0.4'].loc[latest_touch_time]
dfs4

In [ ]:
dfs4['grade'].value_counts()[0]/dfs4.shape[0]
# correct rate
dfs4['grade'].value_counts()

In [ ]:
#group
num_of_trials = dfgh.groupby('ini_time').ngroups
print('num of trials : ',num_of_trials)

In [ ]:
latest_touch_time= dfgh.groupby('ini_time')['response_time'].idxmax()
dfgj = dfgh.loc[latest_touch_time]
dfgj.shape

In [ ]:

dfgg = dfgh.groupby('ini_time').agg({'response_time':['mean','std'], 'click_x':['mean','std'],'click_y':['mean','std']}).reset_index()

dfgg.columns = ['ini_time','res_time_mean','res_time_std','clickX_mean','clickX_std','clickY_mean','clickY_std']
selected_col = ['ball_now_pos','choice1','choice2','click_x','click_y','ini_pos','ini_vel','grade','response_time','res_time_mean','res_time_std']
dfa = pd.merge(dfgh,dfgg,on='ini_time')
dfa[selected_col]

In [ ]:
#correct or not
dfc = dfa[dfa['grade']==1]
dff = dfa[dfa['grade']==0]
print(dfc.shape)
print(dff.shape)

In [ ]:

dfc2 = dfa2[dfa2['grade']==1]
dff2 = dfa2[dfa2['grade']==0]
print(dfc2.shape)
print(dff2.shape)

dfc3 = dfa3[dfa3['grade']==1]
dff3 = dfa3[dfa3['grade']==0]
print(dfc3.shape)
print(dff3.shape)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

data1 = dfc2['response_time']
data2 = dff2['response_time']
data3 = dfc3['response_time']
data4 = dff3['response_time']
fig = plt.figure(figsize =(5, 4))
ax = fig.add_axes([0, 0, 1, 1])
x2=[1,2]
x3 = [1.5,2.5]
ax.boxplot([data1,data2],positions=x2)
ax.boxplot([data3,data4],positions=x3)
ax.eventplot([data1,data2], orientation="vertical",lineoffsets=x2,linelengths=0.25,color ='blue')
ax.eventplot([data3,data4], orientation="vertical",lineoffsets=x3,linelengths=0.25,color='orange')
plt.ylim(0,10e6)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm,expon, gamma

data = dfc2['response_time']
data_ = dff2['response_time']
data3 = dfc3['response_time']
data3_ = dff3['response_time']

params = norm.fit(data)
mu, sigma = params
print(params)
x = np.linspace(-1*max(data),max(data))
y = norm.pdf(x,loc=mu,scale=sigma)
plt.plot(x,y,'r',label='grade=1')


params_ = norm.fit(data_)
mu_, sigma_ = params_
print(params_)
x_ = np.linspace(-1*max(data_),max(data_))
y_ = norm.pdf(x_,loc=mu_,scale=sigma_)
plt.plot(x_,y_,'g',label='grade=0')


params3 = norm.fit(data3)
mu3, sigma3 = params3
print(params3)
x3 = np.linspace(-1*max(data3),max(data3))
y3 = norm.pdf(x,loc=mu3,scale=sigma3)
plt.plot(x3,y3,'purple',label='3_grade=1')


params3_ = norm.fit(data3_)
mu3_, sigma3_ = params3_
print(params3_)
x3_ = np.linspace(-1*max(data3_),max(data3_))
y3_ = norm.pdf(x3_,loc=mu3_,scale=sigma3_)
plt.plot(x3_,y3_,'orange',label='3_grade=0')

plt.legend()
plt.xlim(-1e9,1e)
plt.show()

In [ ]:
dfa2[['ini_pos_x','ini_pos_y']]=dfa2['ini_pos'].str.strip('()').str.split(',', expand=True)
columns_to_convert = ['ini_pos_x','ini_pos_y']
dfa2[columns_to_convert] = dfa2[columns_to_convert].apply(pd.to_numeric)

ax1 = plt.subplot(221)
ax1.set_title('dfa2')
dfa2.plot.scatter(x='ini_pos_y', y='res_time_mean', alpha=0.5,ax=ax1)
ax2 = plt.subplot(223)
ax2.set_title('dfa2_zoom')
dfa2.plot.scatter(x='ini_pos_y', y='res_time_mean', alpha=0.5,ax=ax2)
plt.ylim(0,0.2e8)

dfa3[['ini_pos_x','ini_pos_y']]=dfa3['ini_pos'].str.strip('()').str.split(',', expand=True)
columns_to_convert = ['ini_pos_x','ini_pos_y']
dfa3[columns_to_convert] = dfa3[columns_to_convert].apply(pd.to_numeric)

ax3 = plt.subplot(222)
ax3.set_title('dfa3')
dfa3.plot.scatter(x='ini_pos_y', y='res_time_mean', alpha=0.5,ax=ax3)
ax4 = plt.subplot(224)
ax4.set_title('dfa3_zoom')
dfa3.plot.scatter(x='ini_pos_y', y='res_time_mean', alpha=0.5,ax=ax4)
plt.ylim(0,0.2e8)

plt.tight_layout()
plt.show()

In [ ]:
dfa2[['tar_x','tar_y']]=dfa2['choice1'].str.strip('()').str.split(',', expand=True)
columns_to_convert = ['tar_x','tar_y']
dfa2[columns_to_convert] = dfa2[columns_to_convert].apply(pd.to_numeric)

dfa3[['tar_x','tar_y']]=dfa3['choice1'].str.strip('()').str.split(',', expand=True)
columns_to_convert = ['tar_x','tar_y']
dfa3[columns_to_convert] = dfa3[columns_to_convert].apply(pd.to_numeric)


ax1 = plt.subplot(221)
ax1.set_title('dfa2')
dfa2.plot.scatter(x='tar_x', y='res_time_mean', alpha=0.5,ax=ax1)
ax2 = plt.subplot(223)
ax1.set_title('dfa2_zoom')
dfa2.plot.scatter(x='tar_x', y='res_time_mean', alpha=0.5,ax=ax2)
plt.ylim(0,0.2e8)


ax3 = plt.subplot(222)
ax3.set_title('dfa3')
dfa3.plot.scatter(x='tar_x', y='res_time_mean', alpha=0.5,ax=ax3)
ax4 = plt.subplot(224)
ax4.set_title('dfa3_zoom')
dfa3.plot.scatter(x='tar_x', y='res_time_mean', alpha=0.5,ax=ax4)
plt.ylim(0,0.2e8)

plt.tight_layout()
plt.show()

In [ ]:
dfa2['tar_ini_distance'] =dfa2['ini_x']-dfa2['tar_x']
dfa2['tar_ini_distance'] = dfa2['tar_ini_distance'].apply(pd.to_numeric).abs()
dfa3['tar_ini_distance'] =dfa3['ini_x']-dfa3['tar_x']
dfa3['tar_ini_distance'] = dfa3['tar_ini_distance'].apply(pd.to_numeric).abs()

ax1 = plt.subplot(2,2,1)
ax1.set_title('dfa2')
dfa2.plot.scatter(x='tar_ini_distance', y='res_time_mean', alpha=0.5,ax=ax1)

ax2 = plt.subplot(2,2,3)
ax2.set_title('dfa2_zoom')
dfa2.plot.scatter(x='tar_ini_distance', y='res_time_mean', alpha=0.5,ax=ax2)
plt.ylim(0,1e7)

ax3 = plt.subplot(2,2,2)
ax3.set_title('dfa3')
dfa3.plot.scatter(x='tar_ini_distance', y='res_time_mean', alpha=0.5,ax=ax3)

ax4 = plt.subplot(2,2,4)
ax3.set_title('dfa3_zoom')
dfa3.plot.scatter(x='tar_ini_distance', y='res_time_mean', alpha=0.5,ax=ax4)
plt.ylim(0,1e7)

plt.tight_layout()
plt.show()